Vektor Model Embedding

In [ ]:
#jalankan library
!pip install contractions
!pip install textsearch
!pip install tqdm
!!pip install nltk
import nltk
!pip instal difflib
nltk.download('punkt')

ERROR: unknown command "instal" - maybe you meant "install"


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Panggil library
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

Sinkron GdDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Panggil KeyVector dari Gensim

In [ ]:
# KeyedVector berguna untuk menampung vektor siap pakai berukuran kecil dari word embedding model
from gensim.models.keyedvectors import KeyedVectors

import time

jalankan dengan waktu lebih dari 1 menit

In [ ]:
waktu_mulai = time.time()

# File vektor didapat dari https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.id.vec
file_vektor_dari_model = '/content/drive/MyDrive/1167050002/wiki.id.vec'
vektor_dari_kata = KeyedVectors.load_word2vec_format(file_vektor_dari_model, limit=0)

waktu_selesai = time.time()
waktu_proses = waktu_selesai - waktu_mulai
print("Waktu proses: ", waktu_proses)

Waktu proses:  72.09161281585693


Input Pertanyaan dan Vector Embedding

In [28]:
import re
from collections import Counter

def words(text):
    return re.findall(r'\w+', text.lower())

input_text = 'harga sewa gunung merapi berapa? '

# Hapus simbol
hapus_simbol = re.sub(r'[^\w]', ' ', input_text)
#penyesuaian kata
kalimat_koreksi = hapus_simbol.lower()

#test kecocokan dan koreksi bahasa indonesia typography
WORDS = Counter(words(open('/content/drive/MyDrive/1167050002/katadasar.txt').read()))

def P(word, N=sum(WORDS.values())):
    return WORDS[word] / N

def correction(word):
    return max(candidates(word), key=P)

def candidates(word):
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    return set(w for w in words if w in WORDS)

def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

kata_koreksi = []
for kata in kalimat_koreksi.split():
    kata_koreksi.append(correction(kata))

hasil_koreksi = ' '.join(kata_koreksi)

print('Input Kalimat:', input_text)
print('Hasil Koreksi:', hasil_koreksi)

# komputasi untuk mendapatkan vektor dari setiap kata
for kata in hasil_koreksi.split():
    if kata in vektor_dari_kata:
        word_embedding_kata = vektor_dari_kata[kata]
        print(word_embedding_kata)


Input Kalimat: harga sewa gunung merapi berapa? 
Hasil Koreksi: harga sewa gunung merapu berapa
[ 8.2924e-02  9.1183e-02  3.5705e-01  3.3147e-01 -2.6619e-01 -1.2007e-02
 -4.8493e-01  1.3673e-02  9.0422e-02 -1.4645e-01 -1.3373e-01 -1.7060e-01
 -2.6030e-02  1.3511e-01  2.5707e-01 -7.3793e-02 -1.4136e-01 -3.8668e-01
 -2.4062e-01  2.8322e-01 -1.1614e-02  4.2609e-01 -4.7855e-01 -5.1059e-02
 -4.4983e-01  1.1604e-01 -1.7221e-01  2.6221e-01 -1.8255e-02  1.1219e-01
  2.5873e-01  6.8955e-02 -5.2243e-01  2.7800e-01 -5.9887e-01 -1.7371e-01
 -6.9614e-02  2.7922e-01 -4.0638e-01 -9.4665e-02  2.0937e-01  5.3154e-02
  1.7427e-01 -1.9941e-01  4.2349e-01 -2.3377e-01  6.9187e-02 -1.8278e-01
 -9.5060e-02 -9.5275e-02  4.4916e-01 -3.7391e-01 -3.7015e-01  2.7414e-01
  7.4977e-02 -6.1915e-01 -4.0313e-01 -5.5390e-02 -2.4938e-01 -3.5661e-01
 -2.5019e-01 -8.4286e-02  1.0009e-02 -3.8373e-02 -4.1673e-01  8.6950e-02
 -1.4803e-01  7.1411e-01  4.1756e-01 -1.3590e-01 -2.0957e-01 -1.7110e-01
  1.3059e-01 -2.0000e-02  1.

Preprocessing Dataset

In [ ]:
# Membaca Dataset
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/1167050002/dataset_super_final.xltx')
data

,label,question,answer
0,carier,Maaf kang carier itu apa?,Carier adalah suatu alat atau tas dengan kapas...
1,carier,Apa daypack itu?,Daypack adalah tempat atau alat yang digunakan...
2,carier,Daypack yang ada kapasitas berapa saja?,Kapasitas daypack hanya 30liter saja.
3,carier,Kang carier yang tersedia tuh kapasitas berapa...,Untuk kapasitas carier yang tersedia dimulai d...
4,carier,Rekomendasi dong kang carier buat perjalanan 1...,Rekomendasi carier untuk perjalan 1-2 adalah c...
...,...,...,...
195,persyaratan,Bisa ga kang pesyaratan yang diserahkan BPKB?,Untuk BPKB belum bisa diserahkan sebagai persy...
196,persyaratan,Bisa ga kang pesyaratan yang diserahkan sertif...,Untuk Sertifikat belum bisa diserahkan sebagai...
197,persyaratan,Bisa ga kang pesyaratan yang diserahkan barang...,Untuk Barang antik belum bisa diserahkan sebag...
198,persyaratan,Bisa ga kang pesyaratan yang diserahkan handph...,Untuk Handphone bisa diserahkan sebagai persya...


In [ ]:
# Jumlah Kelas\Label Pada Dataset
data['label'].value_counts()

peralatan      83
carier         35
tenda          34
persyaratan    19
trekking       15
penerangan     14
Name: label, dtype: int64

In [ ]:
# Encode Kelas Dataset
encoder= LabelEncoder()
encoder.fit(data['label'])
training_labels = encoder.transform(data['label'])

cleaning dan case folding

In [ ]:
# Import required libraries
import re
import pandas as pd
from nltk.tokenize import word_tokenize  # Import the word_tokenize function
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Path to the Excel file on Google Drive
file_path = '/content/drive/MyDrive/1167050002/dataset_super_final.xltx'

# Read data from the Excel file
data = pd.read_excel(file_path)

# kolom pertanyaan untuk diproses terlebih dahulu dan diberi token
target_column = 'question'

# Function to preprocess text (remove symbols and lowercase)
def preprocess_text(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)  # Remove symbols
    folded_text = cleaned_text.lower()  # Convert to lowercase
    return folded_text

# Proses (cleaning dan case-folding) dari pertanyaan
data['cleaned_question'] = data[target_column].apply(preprocess_text)

# Tampilan Hasil Preprocessing (cleaned_question)
print("Hasil Cleaning dan Case-Folding")
print(data[['question', 'cleaned_question']])

# Tokenization and sequence encoding configuration
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(data[target_column])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(data[target_column])
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# print("\nHasil Tokenizing dan Pengkodean Urutan:")
# print(padded_sequences)


Hasil Cleaning dan Case-Folding
                                              question  \
0                            Maaf kang carier itu apa?   
1                                     Apa daypack itu?   
2              Daypack yang ada kapasitas berapa saja?   
3    Kang carier yang tersedia tuh kapasitas berapa...   
4    Rekomendasi dong kang carier buat perjalanan 1...   
..                                                 ...   
195      Bisa ga kang pesyaratan yang diserahkan BPKB?   
196  Bisa ga kang pesyaratan yang diserahkan sertif...   
197  Bisa ga kang pesyaratan yang diserahkan barang...   
198  Bisa ga kang pesyaratan yang diserahkan handph...   
199    Bisa ga kang pesyaratan yang diserahkan laptop?   

                                      cleaned_question  
0                             maaf kang carier itu apa  
1                                      apa daypack itu  
2               daypack yang ada kapasitas berapa saja  
3    kang carier yang tersedia tuh kapasita

Tokenizing

In [ ]:
#Tokenizing
# Function to tokenize text using NLTK's word_tokenize
def tokenize_text(text):
    tokens = word_tokenize(text)  # Tokenize words
    return tokens

# Proses Tokenizing dari kolom pertanyaan
data['tokenized_question'] = data['cleaned_question'].apply(tokenize_text)

# Tampilan Hasil Tokenizing (tokenized_question)
print("Hasil Tokenizing")
print(data[['cleaned_question', 'tokenized_question']])


Hasil Tokenizing
                                      cleaned_question  \
0                             maaf kang carier itu apa   
1                                      apa daypack itu   
2               daypack yang ada kapasitas berapa saja   
3    kang carier yang tersedia tuh kapasitas berapa...   
4    rekomendasi dong kang carier buat perjalanan 1...   
..                                                 ...   
195       bisa ga kang pesyaratan yang diserahkan bpkb   
196  bisa ga kang pesyaratan yang diserahkan sertif...   
197  bisa ga kang pesyaratan yang diserahkan barang...   
198  bisa ga kang pesyaratan yang diserahkan handphone   
199     bisa ga kang pesyaratan yang diserahkan laptop   

                                    tokenized_question  
0                       [maaf, kang, carier, itu, apa]  
1                                  [apa, daypack, itu]  
2        [daypack, yang, ada, kapasitas, berapa, saja]  
3    [kang, carier, yang, tersedia, tuh, kapasitas,...  
4

Konversi Dataset Ke Tokenizing

In [ ]:
# vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = ""

tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(data.question)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(data.question)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

Model CNN

In [29]:
!pip install wandb
from wandb.keras import WandbCallback
import wandb
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

# Langkah 1: Inisialisasi WandB run
wandb.init(project='cnn')

# Tentukan konstanta
vocab_size = 10000  # Ganti dengan ukuran vocabulary
EMBED_SIZE = 300
MAX_SEQUENCE_LENGTH = 20

# Langkah 2: Simpan input model dan hyperparameter
config = wandb.config
config.learning_rate = 0.001

# Buat model CNN
model = Sequential()
model.add(Embedding(vocab_size, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(BatchNormalization())
# Import BatchNormalization from keras.layers
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(BatchNormalization())
# Import BatchNormalization from keras.layers
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(BatchNormalization())
# Import BatchNormalization from keras.layers
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# latih model Pembelajaran CNN
def lr_schedule(epoch):
    initial_learning_rate = 0.001
    decay_steps = 10000
    decay_rate = 0.9
    if epoch < 50:
        return initial_learning_rate
    else:
        return initial_learning_rate * decay_rate ** (epoch // decay_steps)

lr_scheduler = LearningRateScheduler(lr_schedule)

# Komputasi model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Latih model
model.summary()

num_samples = 1000
vocab_size = 10000
MAX_SEQUENCE_LENGTH = 20

# sumbu x dan y
import numpy as np
X = np.random.randint(1, vocab_size, size=(10000, MAX_SEQUENCE_LENGTH))
y = np.random.randint(1, 2, size=(200,))


accuracy,▃▆▄█▅▃▅▅▃▅▅█▃▆▅▆▄▃▃▅▄▄▄▆▂▅▅▂▄▅▃▄▄▅▃▅▄▂▁▄
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███▁▁▁▂
loss,███████████████▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▁
accuracy,0.125
epoch,49
loss,-6974151.5


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 300)           3000000   
                                                                 
 conv1d_3 (Conv1D)           (None, 20, 128)           115328    
                                                                 
 batch_normalization_3 (Batc  (None, 20, 128)          512       
 hNormalization)                                                 
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 10, 128)          0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 10, 64)            24640     
                                                                 
 batch_normalization_4 (Batc  (None, 10, 64)          

Uji Model CNN

In [36]:
# Langkah 3: Uji model CNN dengan WandB Callback
epochs = 40
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs, callbacks=[WandbCallback()])

Epoch 1/40
7/7 [==============================] - 0s 58ms/step - loss: -25219.4219 - accuracy: 0.1750
Epoch 2/40
7/7 [==============================] - 0s 69ms/step - loss: -25990.5859 - accuracy: 0.2000
Epoch 3/40
7/7 [==============================] - 1s 105ms/step - loss: -27068.8516 - accuracy: 0.1400
Epoch 4/40
7/7 [==============================] - 1s 105ms/step - loss: -28041.5684 - accuracy: 0.1400
Epoch 5/40
7/7 [==============================] - 1s 105ms/step - loss: -28970.8926 - accuracy: 0.2300
Epoch 6/40
7/7 [==============================] - 1s 87ms/step - loss: -30221.4258 - accuracy: 0.1250
Epoch 7/40
7/7 [==============================] - 1s 105ms/step - loss: -31273.1426 - accuracy: 0.1750
Epoch 8/40
7/7 [==============================] - 1s 106ms/step - loss: -32476.7109 - accuracy: 0.1300
Epoch 9/40
7/7 [==============================] - 1s 107ms/step - loss: -33335.4492 - accuracy: 0.2100
Epoch 10/40
7/7 [==============================] - 1s 112ms/step - loss: -34

Dapatkan Jawaban dengan Cosine Similarity

In [33]:
from difflib import SequenceMatcher

data['skor'] = data['question'].apply(lambda row: SequenceMatcher(None, hasil_koreksi, row).ratio())

# Find the index of the most cosine similar question
most_similar_idx = data['skor'].idxmax()

# Extract the predicted label based on the most similar question
predicted_tag = data.loc[most_similar_idx, 'label']

# Filter responses based on the predicted tag
response = data[data['label'] == predicted_tag]

# Find the response with the highest cosine similarity score
tes = response.loc[response['skor'].idxmax()]

print("OUTPUT")
print("Predicted Kelas:", predicted_tag)
print("Predicted jawaban:", tes.answer)



OUTPUT
Predicted Kelas: peralatan
Predicted jawaban: Harga sewa Jaket Gunung adalah 12k/hari.
